In [2]:
import os
import exifread
import pandas as pd

path = r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\Artifical Vision\Data_AlbertoCole"


Paso 1: Extraer metadatos GPS de las imágenes y guardarlos en un CSV

In [2]:
rows = []
for file in os.listdir(path):
    if file.lower().endswith(".jpg"):
        with open(os.path.join(path, file), 'rb') as f:
            tags = exifread.process_file(f)
        try:
            lat = tags['GPS GPSLatitude'].values
            lon = tags['GPS GPSLongitude'].values
            alt = tags['GPS GPSAltitude'].values[0].num / tags['GPS GPSAltitude'].values[0].den

            def dms_to_dd(dms):
                d, m, s = [v.num / v.den for v in dms]
                return d + (m / 60.0) + (s / 3600.0)

            rows.append({
                "file": file,
                "lat": dms_to_dd(lat),
                "lon": dms_to_dd(lon),
                "alt": alt
            })
        except KeyError:
            print(f"⚠️ Sin datos GPS: {file}")

df = pd.DataFrame(rows)
df.to_csv("photo_gps_data.csv", index=False)
print(df.head())


Possibly corrupted field Tag 0x2900 in MakerNote IFD
Possibly corrupted field Tag 0xD800 in MakerNote IFD
Possibly corrupted field Tag 0xC500 in MakerNote IFD
Possibly corrupted field Tag 0xE300 in MakerNote IFD
Possibly corrupted field Tag 0xF200 in MakerNote IFD
Possibly corrupted field Tag 0xA500 in MakerNote IFD
Possibly corrupted field Tag 0xD100 in MakerNote IFD
Possibly corrupted field Tag 0x4600 in MakerNote IFD
Possibly corrupted field Tag 0xAA00 in MakerNote IFD
Possibly corrupted field Tag 0x6600 in MakerNote IFD
Possibly corrupted field Tag 0x2D00 in MakerNote IFD
Possibly corrupted field Tag 0x7300 in MakerNote IFD
Possibly corrupted field Tag 0x0167 in MakerNote IFD
Possibly corrupted field Tag 0x0000 in MakerNote IFD
Possibly corrupted field Tag 0x0000 in MakerNote IFD
Possibly corrupted field Tag 0x0000 in MakerNote IFD
Possibly corrupted field Tag 0x016F in MakerNote IFD
Possibly corrupted field Tag 0x029F in MakerNote IFD
Possibly corrupted field Tag 0x02A5 in MakerNo

           file       lat        lon   alt
0  DJI_0001.JPG  8.948100  83.455759  50.1
1  DJI_0002.JPG  8.948251  83.455758  50.2
2  DJI_0003.JPG  8.948396  83.455757  50.1
3  DJI_0004.JPG  8.948539  83.455757  50.0
4  DJI_0005.JPG  8.948678  83.455756  49.8


In [6]:
from pyproj import Transformer
from core import geoenv
geoenv.setup_all()


proj.db existe: True
GDAL data dir existe: True
✅ pyproj configurado: C:\Users\HeyCe\.conda\envs\cartografunk\Library\share\proj


In [7]:
df = pd.read_csv("photo_gps_data.csv")

# Definir transformador de WGS84 (EPSG:4326) a UTM zona 17N
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32617", always_xy=True)
df["x"], df["y"] = transformer.transform(df["lon"].values, df["lat"].values)

df[["file", "x", "y", "alt"]].to_csv("photo_gps_utm.csv", index=False)
print(df.head())


           file       lat        lon   alt             x             y
0  DJI_0001.JPG  8.948100  83.455759  50.1  2.229202e+06  1.896964e+07
1  DJI_0002.JPG  8.948251  83.455758  50.2  2.229201e+06  1.896963e+07
2  DJI_0003.JPG  8.948396  83.455757  50.1  2.229200e+06  1.896961e+07
3  DJI_0004.JPG  8.948539  83.455757  50.0  2.229200e+06  1.896959e+07
4  DJI_0005.JPG  8.948678  83.455756  49.8  2.229199e+06  1.896958e+07


In [1]:
import open3d as o3d
import numpy as np

points = df[["x", "y", "alt"]].to_numpy()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Guardar y visualizar
o3d.io.write_point_cloud("photo_positions.ply", pcd)
o3d.visualization.draw_geometries([pcd])


ModuleNotFoundError: No module named 'open3d'